In [45]:
import numpy as np
import pandas as pd
import locale
import time

locale.setlocale(locale.LC_TIME, "de_DE")

%store -r columnNames

#pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', None)  # or 199


# pd.describe_option('display')

# Analyze Data

In [46]:
surveyData=pd.read_csv('justCompletedAttempts.csv', sep=';')
print("Anzahl an kompletten Daten:", len(surveyData))

# Add extra fields for comparison
surveyData['duration-date-based'] = surveyData.apply(lambda row: pd.to_datetime(row['date-last'])-pd.to_datetime(row['date-start']), axis=1)
surveyData['duration-date-based-in-seconds'] = surveyData.apply(lambda row: row['duration-date-based'].total_seconds(), axis=1)
surveyData['duration-delta-all-vs-start-end'] = surveyData.apply(lambda row: row['duration-over-all']-row['duration-date-based-in-seconds'], axis=1)


Anzahl an kompletten Daten: 203


## Time Delta

In [47]:

# Check time delta
greatDelta = 10 
extremeDelta = 20
print()
print(f"Overall länger",len(surveyData.loc[surveyData['duration-delta-all-vs-start-end'] >= 0]))
print(f"Overall sehr viel länger, AKA Start-End schneller >{greatDelta} Sek. schneller:",len(surveyData.loc[surveyData['duration-delta-all-vs-start-end'] >= greatDelta]))
print(f"Overall extrem länger, AKA Start-End schneller >{extremeDelta} Sek. schneller:",len(surveyData.loc[surveyData['duration-delta-all-vs-start-end'] >= extremeDelta]))
print()
print(f"Start-End länger",len(surveyData.loc[surveyData['duration-delta-all-vs-start-end'] < 0]))
print(f"Start-End sehr viel länger, AKA Overall >{greatDelta} Sek. schneller:",len(surveyData.loc[surveyData['duration-delta-all-vs-start-end'] < -greatDelta]))
print(f"Start-End extrem länger, AKA Overall >{extremeDelta} Sek. schneller:",len(surveyData.loc[surveyData['duration-delta-all-vs-start-end'] < -extremeDelta]))

# surveyData[['id', 'duration-over-all', 'duration-date-based-in-seconds', 'duration-delta-all-vs-start-end', 'duration-date-based']]


Overall länger 134
Overall sehr viel länger, AKA Start-End schneller >10 Sek. schneller: 10
Overall extrem länger, AKA Start-End schneller >20 Sek. schneller: 9

Start-End länger 69
Start-End sehr viel länger, AKA Overall >10 Sek. schneller: 20
Start-End extrem länger, AKA Overall >20 Sek. schneller: 16


## Building Groups

In [48]:
# Build groups depending on what viz participant had to solve first

listFirstParticipants=surveyData[surveyData['list-question-number']==6]

chordFirstParticipants=surveyData[surveyData['chord-question-number']==6]

mapFirstParticipants=surveyData[surveyData['map-question-number']==6]

print("Liste zu erst", len(listFirstParticipants))
print("Chord zu erst", len(chordFirstParticipants))
print("Map zu erst", len(mapFirstParticipants))

Liste zu erst 52
Chord zu erst 78
Map zu erst 73


### Define Evaluation Columns

In [49]:
# Build evaluation columns 

evalColumns = [
    'id', 
    'date-start', 
    'date-last',
    'duration-over-all',
    'duration-date-based-in-seconds', 
    'duration-group-tasks-intro',
    'duration-group-list-task',
    'duration-group-chord-task',
    'duration-group-map-task',
    'age', 
    'sex', 
    'internet-level', 
    'mood-start', 
    'mood-end',
    'list-question-number',
    'chord-question-number',
    'map-question-number',
    'preference-chord-or-map-over-list', 
    'highest-preferenced-viz', 
    'preference-reason', 
    'qualitative-questions-pros-cons-chord', 
    'qualitative-questions-pros-cons-map', 
    'qualitative-questions-pros-cons-list'
]

### Helper Functions

In [50]:
def getIndexLine(row):
    indexLine = f"ID: {row.id}" + " | " + f"Alter: {int(row.age)}" +  " | "
    indexLine += f"Geschlecht: { getSex(row.sex) }" + " | " + f"I-Level: { getInternetLevel(row['internet-level']) }" + " | "
    indexLine += f"Start: { row['mood-start'] }" + " | " + f"Ende: { row['mood-end'] }" + " | " +  f"Abschluss: {row['date-last']}"
    return indexLine

# Get gender
def getSex(sex):
    if sex == "männlich":
        return "♂"
    elif sex == "weiblich":
        return "♀"
    else: 
        return "X"

# Get interne level
def getInternetLevel(iLevel):
    if iLevel == "Internet-Profi":
        return "Pro"
    elif iLevel == "Erfahrene Nutzerin, Erfahrener Nutzer":
        return "Middle"
    elif iLevel == "Anfänger":
        return "Low"

# Get viz duration 
def getVizDuration(vizType, row):
    if vizType == "Liste":
        return row['duration-group-list-task']
    elif vizType == "Chord":
        return row['duration-group-chord-task']
    elif vizType == "Map":
        return row['duration-group-map-task']

# Convert Seconds
def secondsToTime(seconds): 
    timeString =  str( time.strftime('%H:%M:%S', time.gmtime(seconds)) ) + str( seconds )[-3:]
    return timeString

# Get viz order
def getVizOrder(row):
    vizQuestionNumbers = {'Liste': row['list-question-number'], 'Chord': row['chord-question-number'], 'Map': row['map-question-number']}        

    sort_viz = sorted(vizQuestionNumbers.items(), key=lambda x: x[1])

    vizOrder = {}
    for i in sort_viz:
        vizOrder.update({i[0]: i[1]})
    
    # orderString = ', '.join(vizOrder)
    iterations = 0
    orderString = ''
    for viz in vizOrder.keys(): 
        iterations += 1
        orderString += viz +  " "
        # orderString +=  str( time.strftime('%H:%M:%S', time.gmtime(getVizDuration(viz, row))) ) + str( getVizDuration(viz, row) )[-3:]
        orderString +=  secondsToTime( getVizDuration(viz, row) )
        if(iterations < 3): orderString += ", "

    return orderString

# Get viz order line with durations
def getOrderedDurationLine(row): 
    shorterAll = min([ row['duration-over-all'], row['duration-date-based-in-seconds'] ])
    orderedDurationLine = "Task-Reihenfolge: " +  getVizOrder(row) + " | " 
    orderedDurationLine += "Video: " + secondsToTime(row['duration-group-tasks-intro']) + " | " 
    orderedDurationLine += "Gesamt: " + secondsToTime(shorterAll)
    return orderedDurationLine

# Get preferences 
def getPreferencesLine(row):
    preferencesLine = "Chord oder Map Präferenz: " + row['preference-chord-or-map-over-list'] + " | " 
    preferencesLine += "Favorisiert: " + row['highest-preferenced-viz']
    return preferencesLine



## Write Out Data

### Combined Data

In [51]:
# Write data to analyze quality feedback combined

for index, row in surveyData.iterrows():
    # Create standard line
    print(getIndexLine(row))
    # Create viz order line with durations
    print(getOrderedDurationLine(row))
    # Create decision line
    print(getPreferencesLine(row))
    # Reason for preference
    print(row['preference-chord-or-map-over-list'], "← Präferenzgrund Chord/Map:")
    print(row['preference-reason'])
    # Pros Cons List
    print('Liste Pro/Contra')
    print(row['qualitative-questions-pros-cons-list']) 
    # Pros Cons Chord
    print('Chord Pro/Contra')
    print(row['qualitative-questions-pros-cons-chord']) 
    # Pros Cons Map
    print('Maps Pro/Contra')
    print(row['qualitative-questions-pros-cons-map']) 
    print()

ID: 10 | Alter: 21 | Geschlecht: ♂ | I-Level: Middle | Start: Gut | Ende: Gut | Abschluss: 2022-04-07 10:25:05
Task-Reihenfolge: Liste 00:02:42.66, Map 00:01:20.64, Chord 00:02:344.7 | Video: 00:12:51.95 | Gesamt: 00:37:25.25
Chord oder Map Präferenz: Ja | Favorisiert: Map
Ja ← Präferenzgrund Chord/Map:
bildliche Veranschaulichung, besserer Überblick
Liste Pro/Contra
Sah alles sehr ähnlich aus, schwierig die Unterschiede fest zu machen
Chord Pro/Contra
teilweise schwierig den Überblick zu behalten aufgrund von Unübersichtlichkeit, viel Text auf wenig Raum
Maps Pro/Contra
Übersichtlich, aufgrund der Satellitenansicht lässt sich ein guter Überblick verschaffen

ID: 11 | Alter: 21 | Geschlecht: ♂ | I-Level: Pro | Start: Nicht gut | Ende: Nicht gut | Abschluss: 2022-04-07 10:25:02
Task-Reihenfolge: Chord 00:03:24.66, Liste 00:01:40.81, Map 00:02:17.97 | Video: 00:07:02.98 | Gesamt: 00:36:188.0
Chord oder Map Präferenz: Ja | Favorisiert: Map
Ja ← Präferenzgrund Chord/Map:
Sieht besser aus
L

### List Preferred 

In [52]:
# Write list preferred data
for index, row in surveyData[surveyData['highest-preferenced-viz'] == "Liste"].iterrows():
    # Create standard line
    print(getIndexLine(row))
    # Create viz order line with durations
    print(getOrderedDurationLine(row))
    # Create decision line
    print(getPreferencesLine(row))
    # Reason for preference
    print(row['preference-chord-or-map-over-list'], "← Präferenzgrund Chord/Map:")
    print(row['preference-reason'])
    # Pros Cons List
    print('Liste Pro/Contra')
    print(row['qualitative-questions-pros-cons-list']) 
    # Pros Cons Chord
    print('Chord Pro/Contra')
    print(row['qualitative-questions-pros-cons-chord']) 
    # Pros Cons Map
    print('Maps Pro/Contra')
    print(row['qualitative-questions-pros-cons-map']) 
    print()

ID: 23 | Alter: 20 | Geschlecht: ♂ | I-Level: Middle | Start: Neutral | Ende: Neutral | Abschluss: 2022-04-07 10:33:24
Task-Reihenfolge: Liste 00:04:14.27, Chord 00:02:55.67, Map 00:03:11.18 | Video: 00:08:23.26 | Gesamt: 00:33:033.0
Chord oder Map Präferenz: Nein | Favorisiert: Liste
Nein ← Präferenzgrund Chord/Map:
Unnötig komplexe Bedienung
Liste Pro/Contra
Simpel und verständlich für jeden, aber wichtige Infos gehen verloren
Chord Pro/Contra
Navigieren manchmal schwierig aufgrund des kleinen Interfaces, zu viele Kategorien
Maps Pro/Contra
Geografische Einordnung besser, aber Navigation mit den Kategorienfenstern umständlich beim zoomen

ID: 26 | Alter: 24 | Geschlecht: ♂ | I-Level: Pro | Start: Gut | Ende: Sehr gut | Abschluss: 2022-04-07 10:53:00
Task-Reihenfolge: Liste 00:04:25.95, Map 00:05:44.47, Chord 00:04:13.24 | Video: 00:09:43.12 | Gesamt: 00:51:588.0
Chord oder Map Präferenz: Nein | Favorisiert: Liste
Nein ← Präferenzgrund Chord/Map:
Das Chord-Diagramm ist recht unübersic

### Chord Preferred

In [53]:
# Write chord preferred data
for index, row in surveyData[surveyData['highest-preferenced-viz'] == "Chord"].iterrows():
    # Create standard line
    print(getIndexLine(row))
    # Create viz order line with durations
    print(getOrderedDurationLine(row))
    # Create decision line
    print(getPreferencesLine(row))
    # Reason for preference
    print(row['preference-chord-or-map-over-list'], "← Präferenzgrund Chord/Map:")
    print(row['preference-reason'])
    # Pros Cons List
    print('Liste Pro/Contra')
    print(row['qualitative-questions-pros-cons-list']) 
    # Pros Cons Chord
    print('Chord Pro/Contra')
    print(row['qualitative-questions-pros-cons-chord']) 
    # Pros Cons Map
    print('Maps Pro/Contra')
    print(row['qualitative-questions-pros-cons-map']) 
    print()

ID: 13 | Alter: 20 | Geschlecht: ♂ | I-Level: Middle | Start: Gut | Ende: Gut | Abschluss: 2022-04-07 10:38:17
Task-Reihenfolge: Map 00:05:27.75, Liste 00:03:36.65, Chord 00:02:12.03 | Video: 00:08:37.63 | Gesamt: 00:39:322.0
Chord oder Map Präferenz: Ja | Favorisiert: Chord
Ja ← Präferenzgrund Chord/Map:
Ich denke die Chord-Ansicht ist mit Abstand die übersichtlichste der drei Ansichten...
Meine Meinung nach ist die Karten-Ansicht nur dazu zugebrauchen, um die Lage einschätzen zu können
Liste Pro/Contra
+Auch beispielweise am Handy gut darstellbar und erkennbar
-Vergleichen auch schwierig, da man die Verbindungen nicht so gut erkennt..
Chord Pro/Contra
+Die Übersichtlichkeit
+Alles auf einen Blick
-auf kleinen Bildschirmen sehr klein
-Bei sehr vielen 6+ Wirtschaften sehr unübersichtlich... max. 4-5 würde ich sagen
Maps Pro/Contra
-um details zu vergleichen sehr unübersichtlich... 
- Felder überlagern sich, muss man erst ausseinander ziehen...
+um die Lage einzuschätzen sehr gut 


ID:

### Map Preferred

In [54]:
# Write map preferred data
for index, row in surveyData[surveyData['highest-preferenced-viz'] == "Map"].iterrows():
    # Create standard line
    print(getIndexLine(row))
    # Create viz order line with durations
    print(getOrderedDurationLine(row))
    # Create decision line
    print(getPreferencesLine(row))
    # Reason for preference
    print(row['preference-chord-or-map-over-list'], "← Präferenzgrund Chord/Map:")
    print(row['preference-reason'])
    # Pros Cons List
    print('Liste Pro/Contra')
    print(row['qualitative-questions-pros-cons-list']) 
    # Pros Cons Chord
    print('Chord Pro/Contra')
    print(row['qualitative-questions-pros-cons-chord']) 
    # Pros Cons Map
    print('Maps Pro/Contra')
    print(row['qualitative-questions-pros-cons-map']) 
    print()

ID: 10 | Alter: 21 | Geschlecht: ♂ | I-Level: Middle | Start: Gut | Ende: Gut | Abschluss: 2022-04-07 10:25:05
Task-Reihenfolge: Liste 00:02:42.66, Map 00:01:20.64, Chord 00:02:344.7 | Video: 00:12:51.95 | Gesamt: 00:37:25.25
Chord oder Map Präferenz: Ja | Favorisiert: Map
Ja ← Präferenzgrund Chord/Map:
bildliche Veranschaulichung, besserer Überblick
Liste Pro/Contra
Sah alles sehr ähnlich aus, schwierig die Unterschiede fest zu machen
Chord Pro/Contra
teilweise schwierig den Überblick zu behalten aufgrund von Unübersichtlichkeit, viel Text auf wenig Raum
Maps Pro/Contra
Übersichtlich, aufgrund der Satellitenansicht lässt sich ein guter Überblick verschaffen

ID: 11 | Alter: 21 | Geschlecht: ♂ | I-Level: Pro | Start: Nicht gut | Ende: Nicht gut | Abschluss: 2022-04-07 10:25:02
Task-Reihenfolge: Chord 00:03:24.66, Liste 00:01:40.81, Map 00:02:17.97 | Video: 00:07:02.98 | Gesamt: 00:36:188.0
Chord oder Map Präferenz: Ja | Favorisiert: Map
Ja ← Präferenzgrund Chord/Map:
Sieht besser aus
L